In [2]:
import numpy as np
import pickle as pc

from music21 import instrument, stream, note, chord

from keras.models import Sequential
from keras.layers import Activation
from keras.layers import Dropout
from keras.layers import Dense
from keras.layers import BatchNormalization
from keras.layers import LSTM

In [3]:
def seqGen(notes, pitchNames, vocab):
    noteEnum1 = dict((note, num) for num, note in enumerate(pitchNames))
    
    seqLen = 100
    networkIN = []
    networkOUT = []
    
    for i in range(0, len(notes) - seqLen, 1):
        seqIn = notes[i : i+seqLen]
        seqOut = notes[i + seqLen]
        networkIN.append([noteEnum1[ch] for ch in seqIn])
        networkOUT.append(noteEnum1[seqOut])
    patterns = len(networkIN)
    
    normalizedIN = np.reshape(networkIN, (patterns, seqLen, 1))
    normalizedIN = normalizedIN / float(vocab)
    
    return(networkIN, normalizedIN)

In [4]:
def neuralNet(netIN, vocab):
    
    model = Sequential()
    model.add(LSTM(512, input_shape=(netIN.shape[1], netIN.shape[2]), recurrent_dropout=0.2, return_sequences=True))
    model.add(LSTM(512, return_sequences=True, recurrent_dropout=0.2,))
    model.add(LSTM(512))
    model.add(BatchNormalization())
    model.add(Dropout(0.2))
    model.add(Dense(256))
    model.add(Activation('relu'))
    model.add(BatchNormalization())
    model.add(Dropout(0.2))
    model.add(Dense(128))
    model.add(Activation(('relu')))
    model.add(BatchNormalization())
    model.add(Dropout(0.2))
    model.add(Dense(vocab))
    model.add(Activation('softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='rmsprop')
    
    model.load_weights('weights.hdf5')
    return model    

In [5]:
def genNotes(model, netIN, pitchName, vocab):
    begin = np.random.randint(0, len(netIN)-1)
    noteEnum2 = dict((num, note) for num, note in enumerate(pitchName))
    print(noteEnum2)
    pattern = netIN[begin]
    predictOut= []
    
    for note in range(1000):
        prediction = np.reshape(pattern, (1, len(pattern), 1))
        prediction = prediction / float(vocab)
        predictionResult = model.predict(prediction, verbose = 0)
        ind = np.argmax(predictionResult)
        result = noteEnum2[ind]
        predictOut.append(result)
        pattern.append(ind)
        pattern = pattern[1:len(pattern)]
    return predictOut

In [6]:
def generateMusicFile(notes):
    
    res = []
    offset = 0
    
    for singleNote in notes:
        if ('.' in singleNote) or singleNote.isdigit():
            chord = singleNote.split('.')
            music = []
            for nt in chord:
                n = note.Note(int(nt))
                n.storedInstrument = instrument.Piano()
                music.append(n)
            newChord = chord.Chord(music)
            newChord.offset = offset
            res.append(newChord)
        else:
            newNote = note.Note(singleNote)
            newNote.offset = offset
            newNote.storedInstrument = instrument.Piano()
            res.append(newNote)
        offset += 0.5
    fileStream = stream.Stream(res)
    file.write('midi', fp='music.mid')

In [ ]:
with open('data/notes', 'rb') as filePath:
    notes = pc.load(filePath)

pitchNames = sorted(set(note for note in notes))
vocab = len(set(notes))

netIn, normalIn = seqGen(notes, pitchNames, vocab) 
model = neuralNet(normalIn, vocab)
predictionRes = genNotes(model, netIn, pitchNames, vocab)
generateMusicFile(predictionRes)

{0: '0', 1: '0.1.3.6', 2: '0.1.4.8', 3: '0.3', 4: '0.4', 5: '0.4.7', 6: '0.5', 7: '1', 8: '1.3.4.8', 9: '1.3.6', 10: '1.3.6.8', 11: '1.4', 12: '1.4.6.8', 13: '1.4.8', 14: '1.5', 15: '1.5.8', 16: '1.6', 17: '1.7', 18: '10', 19: '10.0', 20: '10.1', 21: '10.1.4', 22: '10.11.3.6', 23: '10.2', 24: '10.2.5', 25: '10.3', 26: '11', 27: '11.0.3.6', 28: '11.1', 29: '11.1.3.6', 30: '11.1.5', 31: '11.2', 32: '11.2.3.6', 33: '11.3', 34: '11.3.4.6', 35: '11.3.5.6', 36: '11.3.6', 37: '11.3.6.7', 38: '11.4', 39: '2', 40: '2.3.5', 41: '2.4.6.7', 42: '2.5', 43: '2.5.7', 44: '2.6', 45: '2.6.9', 46: '2.7', 47: '2.8', 48: '3', 49: '3.5.8.11', 50: '3.5.9', 51: '3.6', 52: '3.6.8', 53: '3.6.8.11', 54: '3.6.9', 55: '3.6.9.11', 56: '3.7', 57: '3.8', 58: '4', 59: '4.6', 60: '4.6.8', 61: '4.7', 62: '4.8', 63: '4.8.11', 64: '4.9', 65: '5', 66: '5.10', 67: '5.11', 68: '5.6.8', 69: '5.6.8.11', 70: '5.8', 71: '5.8.0', 72: '5.8.10.11', 73: '5.8.11', 74: '5.8.11.0', 75: '5.9', 76: '6', 77: '6.10', 78: '6.10.1', 79: '6.

In [ ]:
import tensorflow as tf
physical_devices = tf.config.list_physical_devices('GPU') 
tf.config.experimental.set_memory_growth(physical_devices[0], True)

In [1]:
import tensorflow as tf

In [2]:
config = tf.ConfigProto()
config.gpu_options.allow_growth = True


AttributeError: module 'tensorflow' has no attribute 'ConfigProto'

In [1]:
import os
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID" 
os.environ["CUDA_VISIBLE_DEVICES"] = ""